# Chess.com API Data Collection Project (September 2025)

## Project Overview

The goal of this project is to collect, process, and analyze data from the Chess.com Public API using Python.
I retrieved the top leaderboard players, gathered their detailed profile and game statistics, and collected all their games played in September 2025.

Official API documentation:
🔗 https://www.chess.com/news/view/published-data-api

## 2. Plan of Work

1. Retrieve leaderboards from Chess.com (top 50 players for each activity category).
2. Extract the list of Ukrainian players from the leaderboards.
3. Using the list of leader usernames, get detailed player profiles and statistics for each one.
4. Using these players’ usernames, fetch all their games from September 2025.
5. Flatten JSON dictionaries (player info inside games) into separate columns for easier analysis.
6. Save all datasets as .csv files.

## 3. Importing Required Libraries

In [1]:
import requests
import pandas as pd
import time
from datetime import datetime, timezone
import ast

## 4. Step 1 — Retrieve Global and Ukrainian Leaderboards

Getting the top-50 players in each Chess.com category (live_blitz, live_bullet, live_rapid, daily)
and filter out those from Ukraine.

In [2]:
# --- Request headers ---
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/141.0.0.0 Safari/537.36"
}

# --- 1️⃣ Get Leaderboards ---
leaderboard_url = "https://api.chess.com/pub/leaderboards"
resp = requests.get(leaderboard_url, headers=headers)
resp.raise_for_status()
data = resp.json()

# --- 2️⃣ Create combined leaderboard table ---
types = ["live_blitz", "live_bullet", "live_rapid", "daily"]
all_leaders = []

for t in types:
    for p in data.get(t, []):
        all_leaders.append({
            "type": t,
            "rank": p.get("rank"),
            "username": p.get("username"),
            "score": p.get("score"),
            "country": p.get("country")
        })

df_all = pd.DataFrame(all_leaders)
df_all.to_csv("chesscom_all_leaders.csv", index=False)

# --- 3️⃣ Filter Ukrainian players ---
ua_leaders = df_all[df_all["country"] == "https://api.chess.com/pub/country/UA"]
ua_leaders.to_csv("chesscom_ua_leaders.csv", index=False)

print(f"All players: {len(df_all)}, Ukrainian: {len(ua_leaders)}")


All players: 200, Ukrainian: 1


## 5. Step 2 — Get Detailed Profile and Statistics for Each Player

The code loop through each username from the leaderboard and query two API endpoints:

/player/{username} → basic profile

/player/{username}/stats → detailed ratings and records

In [3]:
# --- Load usernames from leaderboard ---
df_leaders = pd.read_csv("chesscom_all_leaders.csv")
usernames = df_leaders["username"].unique()

players_data = []

# --- 2️⃣ Collect detailed profile and stats ---
for idx, username in enumerate(usernames, start=1):
    print(f"[{idx}/{len(usernames)}] Processing: {username}")

    # Profile info
    profile_url = f"https://api.chess.com/pub/player/{username}"
    profile_resp = requests.get(profile_url, headers=headers)
    if profile_resp.status_code != 200:
        print(f"❌ Cannot get profile for {username}")
        continue
    profile = profile_resp.json()

    # Game statistics
    stats_url = f"https://api.chess.com/pub/player/{username}/stats"
    stats_resp = requests.get(stats_url, headers=headers)
    if stats_resp.status_code != 200:
        print(f"❌ Cannot get stats for {username}")
        continue
    stats = stats_resp.json()

    # Combine profile + stats
    player_data = {
        "username": username,
        "title": profile.get("title"),
        "country": profile.get("country"),
        "status": profile.get("status"),
        "followers": profile.get("followers"),
        "is_streamer": profile.get("is_streamer", False),
        "twitch_url": profile.get("twitch_url"),
        "youtube_url": profile.get("youtube_url"),
        "joined": datetime.fromtimestamp(profile.get("joined"), tz=timezone.utc) if profile.get("joined") else None,
        "last_online": datetime.fromtimestamp(profile.get("last_online"), tz=timezone.utc) if profile.get("last_online") else None
    }

    # Ratings and records per game type
    game_types = ["chess_blitz", "chess_bullet", "chess_rapid", "chess_daily", "tactics"]
    for gtype in game_types:
        info = stats.get(gtype, {})
        player_data[f"{gtype}_rating"] = info.get("last", {}).get("rating")
        player_data[f"{gtype}_best_rating"] = info.get("best", {}).get("rating")
        record = info.get("record", {})
        player_data[f"{gtype}_wins"] = record.get("win", 0)
        player_data[f"{gtype}_losses"] = record.get("loss", 0)
        player_data[f"{gtype}_draws"] = record.get("draw", 0)

    players_data.append(player_data)
    time.sleep(0.5)

# --- Save to CSV ---
df_players = pd.DataFrame(players_data)
df_players.to_csv("top_players_detailed_info.csv", index=False)
print("✅ Saved detailed player info to 'top_players_detailed_info.csv'")

[1/158] Processing: MagnusCarlsen
[2/158] Processing: Hikaru
[3/158] Processing: Firouzja2003
[4/158] Processing: lachesisQ
[5/158] Processing: DenLaz
[6/158] Processing: LyonBeast
[7/158] Processing: gurelediz
[8/158] Processing: GMWSO
[9/158] Processing: FaustinoOro
[10/158] Processing: mishanick
[11/158] Processing: 0gZPanda
[12/158] Processing: Salem-AR
[13/158] Processing: spicycaterpillar
[14/158] Processing: Mykola-Bortnyk
[15/158] Processing: nihalsarin
[16/158] Processing: GHANDEEVAM2003
[17/158] Processing: ChessFighter_2011
[18/158] Processing: Andreikka
[19/158] Processing: artooon
[20/158] Processing: HansOnTwitch
[21/158] Processing: PursuitOfHappyness2
[22/158] Processing: xiaotong2008
[23/158] Processing: rpragchess
[24/158] Processing: FabianoCaruana
[25/158] Processing: LiamPutnam2008
[26/158] Processing: FairChess_on_YouTube
[27/158] Processing: Polish_fighter3000
[28/158] Processing: BogdanDeac
[29/158] Processing: MichaelRoiz
[30/158] Processing: TRadjabov
[31/158]

## 6. Step 3 — Collect All Games (September 2025)

I use each player’s username to download their game archives for September 2025.

In [4]:
from time import sleep

players_file = "top_players_detailed_info.csv"
output_file = "all_players_games_2025_09.csv"
year = 2025
month = 9
pause_sec = 2

top_players = pd.read_csv(players_file)
player_usernames = top_players["username"].tolist()
print(f"Found {len(player_usernames)} players.")

all_games = []
closed_profiles = []

for i, username in enumerate(player_usernames, 1):
    print(f"[{i}/{len(player_usernames)}] Checking {username}...")
    url = f"https://api.chess.com/pub/player/{username}/games/{year}/{month:02d}"
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            for g in data.get("games", []):
                all_games.append({
                    "username": username,
                    "url": g.get("url"),
                    "white": g.get("white"),
                    "black": g.get("black"),
                    "time_class": g.get("time_class"),
                    "rules": g.get("rules"),
                    "end_time": g.get("end_time"),
                    "rated": g.get("rated"),
                    "white_result": g.get("white_result"),
                    "black_result": g.get("black_result"),
                    "eco": g.get("eco"),
                    "tcn": g.get("tcn")
                })
        else:
            closed_profiles.append(username)
    except Exception as e:
        print(f"❌ Error for {username}: {e}")
        closed_profiles.append(username)
    sleep(pause_sec)

games_df = pd.DataFrame(all_games)
games_df.to_csv(output_file, index=False)
print(f"✅ Saved {len(games_df)} games to '{output_file}'")

if closed_profiles:
    print("\nClosed or unavailable profiles:")
    for u in closed_profiles:
        print(u)


Found 158 players.
[1/158] Checking MagnusCarlsen...
[2/158] Checking Hikaru...
[3/158] Checking Firouzja2003...
[4/158] Checking lachesisQ...
[5/158] Checking DenLaz...
[6/158] Checking LyonBeast...
[7/158] Checking gurelediz...
[8/158] Checking GMWSO...
[9/158] Checking FaustinoOro...
[10/158] Checking mishanick...
[11/158] Checking 0gZPanda...
[12/158] Checking Salem-AR...
[13/158] Checking spicycaterpillar...
[14/158] Checking Mykola-Bortnyk...
[15/158] Checking nihalsarin...
[16/158] Checking GHANDEEVAM2003...
[17/158] Checking ChessFighter_2011...
[18/158] Checking Andreikka...
[19/158] Checking artooon...
[20/158] Checking HansOnTwitch...
[21/158] Checking PursuitOfHappyness2...
[22/158] Checking xiaotong2008...
[23/158] Checking rpragchess...
[24/158] Checking FabianoCaruana...
[25/158] Checking LiamPutnam2008...
[26/158] Checking FairChess_on_YouTube...
[27/158] Checking Polish_fighter3000...
[28/158] Checking BogdanDeac...
[29/158] Checking MichaelRoiz...
[30/158] Checking TR

## 7. Step 4 — Flatten Nested JSON Columns

The original dataset contains two columns (white and black) with nested JSON objects.
I unpack these into separate columns to simplify further analysis.

In [5]:
# --- Load CSV with raw game data ---
games_df = pd.read_csv("all_players_games_2025_09.csv")

# --- Convert JSON strings to dictionaries ---
games_df["white"] = games_df["white"].apply(ast.literal_eval)
games_df["black"] = games_df["black"].apply(ast.literal_eval)

# --- Extract fields for 'white' player ---
for field in ["username", "rating", "result", "uuid", "@id"]:
    games_df[f"white_{field.replace('@', 'api_')}"] = games_df["white"].apply(lambda x: x.get(field))

# --- Extract fields for 'black' player ---
for field in ["username", "rating", "result", "uuid", "@id"]:
    games_df[f"black_{field.replace('@', 'api_')}"] = games_df["black"].apply(lambda x: x.get(field))

# --- Drop original nested columns ---
games_df = games_df.drop(columns=["white", "black"])

# --- Save flattened dataset ---
games_df.to_csv("all_players_games_2025_09_flat.csv", index=False)
print("✅ Flattened data saved as 'all_players_games_2025_09_flat.csv'")

games_df.head()


✅ Flattened data saved as 'all_players_games_2025_09_flat.csv'


,username,url,time_class,rules,end_time,rated,white_result,black_result,eco,tcn,white_username,white_rating,white_uuid,white_api_id,black_username,black_rating,black_uuid,black_api_id
0,MagnusCarlsen,https://www.chess.com/game/live/151138695139,blitz,chess,1756825485,True,win,resigned,https://www.chess.com/openings/Kings-Gambit-Ac...,mC0KnDKDbs2MpF1TdN80FMYQlB92cD78DR07RY70Np5OpR...,MagnusCarlsen,3361,a2761738-b155-11df-8018-000000000000,https://api.chess.com/pub/player/magnuscarlsen,kaanozcan006,2641,8fe1daa8-0e58-11e4-802e-000000000000,https://api.chess.com/pub/player/kaanozcan006
1,MagnusCarlsen,https://www.chess.com/game/live/151139296383,blitz,chess,1756826261,True,resigned,win,https://www.chess.com/openings/Sicilian-Defens...,mCYIgv0SlBIBvBWOkA!Tbs7YiqXPcu6XnvZRfm5Zac90eg...,Antosha_Chekhonte,2804,4558565a-8a2b-11e6-8060-000000000000,https://api.chess.com/pub/player/antosha_chekh...,MagnusCarlsen,3362,a2761738-b155-11df-8018-000000000000,https://api.chess.com/pub/player/magnuscarlsen
2,MagnusCarlsen,https://www.chess.com/game/live/151140489265,blitz,chess,1756827143,True,win,checkmated,https://www.chess.com/openings/Undefined,pF0KkA!Tmu5QiqZRbs2Ufm3Nlt92gv8!vM6EmETEdm7Zcl...,MagnusCarlsen,3363,a2761738-b155-11df-8018-000000000000,https://api.chess.com/pub/player/magnuscarlsen,Genghis_K,2930,eebad3fc-e6a4-11e4-801c-000000000000,https://api.chess.com/pub/player/genghis_k
3,MagnusCarlsen,https://www.chess.com/game/live/151141090575,blitz,chess,1756827774,True,resigned,win,https://www.chess.com/openings/Alapin-Sicilian...,mCYIksZJCJ7JlB2Ugv92blIBfAJ7vB!Teg8!dm5QBQXQlv...,MatthewG-p4p,2949,cfcc23e0-6616-11e6-805a-000000000000,https://api.chess.com/pub/player/matthewg-p4p,MagnusCarlsen,3364,a2761738-b155-11df-8018-000000000000,https://api.chess.com/pub/player/magnuscarlsen
4,MagnusCarlsen,https://www.chess.com/game/live/151141697155,blitz,chess,1756828859,True,agreed,agreed,https://www.chess.com/openings/Sicilian-Defens...,mCYIgvZRfA!Tlt5Qeg2Uks92iy8!cuXPbl6Xfe0SpxRJAH...,MagnusCarlsen,3358,a2761738-b155-11df-8018-000000000000,https://api.chess.com/pub/player/magnuscarlsen,DominguezOnYoutube,3050,568bffe2-5492-11e4-8030-000000000000,https://api.chess.com/pub/player/dominguezonyo...


## 8. Summary

1. Total data sources used: Chess.com Public API

2. Main datasets created:

* chesscom_all_leaders.csv — top 50 players per category

* chesscom_ua_leaders.csv — Ukrainian leaderboard players

* top_players_detailed_info.csv — detailed player stats and profiles

* all_players_games_2025_09.csv — all collected games

* all_players_games_2025_09_flat.csv — flattened game data for analysis

## 9. Next Steps

### 1. **Preparing the data for analysis**

Before analyzing the data:

- review the dataset,  
- clean it from duplicates, incorrect or missing values,  
- create additional columns if needed (e.g., total games played),  
- ensure that data types are correct for SQL queries and visualizations.

---

### 2. **Exploring the leader datasets using SQL**

Given the top-50 players dataset for September 2025 across three activity types (blitz, bullet, rapid), answer the following:

1. Which players had the highest rating in each activity type in September 2025?  
2. Which players had the best results (number of wins) in each activity type?  
3. What is the average number of games played among the top-50 players for each activity type?  
4. Which players show the largest difference between current rating and highest achieved rating?  
5. Which countries are represented among the top-50 players, and how many players are from each country?  
6. Which players are streamers, and how does their rating compare to others in the top-50?  
7. Which players have the most losses in each activity type?  
8. Which players demonstrate the most rating stability (minimal difference between current and best rating)?  
9. Is there a correlation between the number of games played and the current rating in each activity type?  
10. Which players have the fewest games played in the top-50, and does it affect their leaderboard position?

---

### 3. **Visualizing the results in Tableau**

After preparing the tables and executing SQL queries:

- import the results into **Tableau**,  
- create dashboards to visually compare top-50 players by activity type,  
- visualize ratings, wins/losses, stability, and players’ countries,  
- add interactive elements to quickly compare different players and metrics.
